In [1]:
from pyqcu.ascend import qcu
import torch
# lat_x, lat_y, lat_z, lat_t = 16, 16, 8, 8
lat_x, lat_y, lat_z, lat_t = 8, 8, 8, 8
_qcu = qcu(lat_size=[lat_x, lat_y, lat_z, lat_t], dtype_list=[torch.complex128, torch.complex64, torch.complex64, torch.complex32], device_list=[
           torch.device('cpu'), torch.device('cpu'), torch.device('cpu'), torch.device('cpu')], dslash='clover', solver='bistabcg', verbose=False)
_qcu.load()
_qcu.init()

Error: MPI_ERR_COMM: invalid communicator
[MPI Rank 0/1] Using cpu:0
[MPI Rank 0/1] Using cpu:0
[MPI Rank 0/1] Using cpu:0
[MPI Rank 0/1] Using cpu:0
rank 0: Data is loaded from -b.h5 (MPI mode)
rank 0: Data is loaded from -x.h5 (MPI mode)
rank 0: Data is loaded from -x0.h5 (MPI mode)
rank 0: Data is loaded from -U.h5 (MPI mode)
rank 0: Data is loaded from -clover_term.h5 (MPI mode)
self.dof_list:[12, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24]
self.dtype_list:[torch.complex128, torch.complex64, torch.complex64, torch.complex32]
self.device_list:[device(type='cpu'), device(type='cpu'), device(type='cpu'), device(type='cpu')]
[MPI Rank 0/1] Using cpu:0
[MPI Rank 0/1] Using cpu:0
[MPI Rank 0/1] Using cpu:0
[MPI Rank 0/1] Using cpu:0
self.lat_size_list:[[8, 8, 8, 8], [4, 4, 4, 4], [2, 2, 2, 2]]


In [ ]:
b = _qcu.b.clone()
x = _qcu.solve().clone()
b = b.reshape(shape=[12]+list(b.shape[-4:]))
x = x.reshape(shape=[12]+list(x.shape[-4:]))
_qcu.test()
_qcu.mg.plot(
    save_path=f"test.ascend.{_qcu.solver}.{_qcu.device_list[0].type}-np{_qcu.size}-dev60.png")
_qcu.save()

In [ ]:
from pyqcu.ascend.define import *
from pyqcu.ascend.dslash import *
_ = xxxtzyx2pxxxtzyx(input_array=b)
b_e = _[0]
b_o = _[1]
_ = xxxtzyx2pxxxtzyx(input_array=x)
x_e = _[0]
x_o = _[1]

M_minus_list = _qcu.mg.op_list[0].hopping.M_minus_list
M_plus_list = _qcu.mg.op_list[0].hopping.M_plus_list
M = _qcu.mg.op_list[0].sitting.M

M_e_minus_list = []
M_o_minus_list = []
M_e_plus_list = []
M_o_plus_list = []
_ = xxxtzyx2pxxxtzyx(input_array=M)
M_e = _[0]
M_o = _[1]
for ward in range(4):
    _ = xxxtzyx2pxxxtzyx(input_array=M_minus_list[ward])
    M_e_minus_list.append(_[0])
    M_o_minus_list.append(_[1])
    _ = xxxtzyx2pxxxtzyx(input_array=M_plus_list[ward])
    M_e_plus_list.append(_[0])
    M_o_plus_list.append(_[1])


def matvec0(src):
    wilson = wilson_mg()
    dest = torch_einsum(
        "EeTZYX, eTZYX->ETZYX", M, src).clone()
    for ward in range(4):
        dest += wilson.give_wilson_minus(ward=ward,
                                         src=src, hopping=M_minus_list[ward])
        dest += wilson.give_wilson_plus(ward=ward,
                                        src=src, hopping=M_plus_list[ward])
    return dest.clone()


def matvec1(src):
    return _qcu.mg.op_list[0].matvec(src)


def matvec2(src):
    _ = xxxtzyx2pxxxtzyx(input_array=src)
    src_e = _[0]
    src_o = _[1]
    dest_e = torch_einsum(
        "EeTZYX, eTZYX->ETZYX", M_e, src_e).clone()
    dest_o = torch_einsum(
        "EeTZYX, eTZYX->ETZYX", M_o, src_o).clone()

    # dest = pxxxtzyx2xxxtzyx(torch.stack([dest_e, dest_o], dim=0))
    # for ward in range(4):
    #     dest += wilson.give_wilson_minus(ward=ward,
    #                                      src=src, hopping=M_minus_list[ward])
    #     dest += wilson.give_wilson_plus(ward=ward,
    #                                     src=src, hopping=M_plus_list[ward])
    for ward in range(4):
        dest_e += torch_einsum("EeTZYX, eTZYX->ETZYX",
                               M_o_minus_list[ward], src_o).clone()
        dest_o += torch_einsum("EeTZYX, eTZYX->ETZYX",
                               M_e_minus_list[ward], src_e).clone()
        dest_e += torch_einsum("EeTZYX, eTZYX->ETZYX",
                               M_o_plus_list[ward], src_o).clone()
        dest_o += torch_einsum("EeTZYX, eTZYX->ETZYX",
                               M_e_plus_list[ward], src_e).clone()
    dest = pxxxtzyx2xxxtzyx(torch.stack([dest_e, dest_o], dim=0))
    return dest.clone()


print(torch.norm(matvec0(x)-matvec1(x))/torch.norm(matvec0(x)))
print(torch.norm(matvec0(x)-matvec2(x))/torch.norm(matvec0(x)))

tensor(2.6862e-16, dtype=torch.float64)
tensor(1.1920, dtype=torch.float64)


In [ ]:
M_e_plus_list[0].shape

In [ ]:
b_e.shape

In [ ]:
M.shape

In [ ]:
_qcu.mg.op_list[0].matvec